In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px
import sqlite3

In [2]:
source_dir = '/Users/arbailey/natcap/idb/data/work/sargassum/shore_segments'
gpkg = 'shoreline_segments.gpkg'
shore_gpkg = os.path.join(source_dir, gpkg)
print(shore_gpkg)

/Users/arbailey/natcap/idb/data/work/sargassum/shore_segments/shoreline_segments.gpkg


# Summarized Normalized Radiance by Segment & Month

In [3]:
# Import normalized nightime lights layer
slc_norm_source_lyr = 'shoreQR_50m_pts_slc_norm'
slc_norm_gdf = gpd.read_file(shore_gpkg, layer=slc_norm_source_lyr)
print(slc_norm_gdf.crs)

{'init': 'epsg:32616'}


In [4]:
slc_norm_gdf

,cf_cvg,shore_ptid,nearest_y,ntlsrc,ntldate,avg_rad,seg_id,type_geomo,nearest_x,geometry
0,14,qrm_633160,476561.890870,slc,20200201,3.860000,42,artificial,2.266580e+06,POINT (476561.729 2266579.636)
1,14,qrm_633210,476563.641307,slc,20200201,3.860000,42,artificial,2.266530e+06,POINT (476563.715 2266529.997)
2,14,qrm_633260,476573.202228,slc,20200201,3.860000,42,artificial,2.266483e+06,POINT (476573.146 2266483.337)
3,14,qrm_633310,476580.967375,slc,20200201,3.860000,42,artificial,2.266526e+06,POINT (476581.088 2266526.026)
4,14,qrm_633360,476614.205614,slc,20200201,2.430000,42,artificial,2.266506e+06,POINT (476614.346 2266505.674)
...,...,...,...,...,...,...,...,...,...,...
1488094,3,coz_026560,524354.974656,slc,20150601,0.117058,103,mixed mud and sand,2.274303e+06,POINT (524355.153 2274302.888)
1488095,3,coz_026610,524305.269319,slc,20150601,0.117058,103,mixed mud and sand,2.274306e+06,POINT (524305.514 2274305.866)
1488096,3,coz_026660,524255.768657,slc,20150601,0.117058,103,mixed mud and sand,2.274308e+06,POINT (524255.876 2274307.851)
1488097,3,coz_026710,524241.886448,slc,20150601,0.117058,103,mixed mud and sand,2.274355e+06,POINT (524241.977 2274355.504)


In [5]:
# Mean and standard deviation of radiance by Segment and Date
slc_norm_gdf.groupby(['seg_id','ntldate']).agg({'avg_rad': [np.mean, np.std]}).reset_index()

seg_id   ntldate   avg_rad          
                            mean       std
0          1  20150601  2.241121  1.427884
1          1  20150701  2.629657  1.800175
2          1  20150801  2.463786  1.656322
3          1  20150901  2.782790  2.041003
4          1  20151001  4.461700  2.909745
...      ...       ...       ...       ...
11224    197  20191001  6.160526  1.687524
11225    197  20191101  7.576842  1.632779
11226    197  20191201  8.199474  2.272965
11227    197  20200101  9.758421  1.827498
11228    197  20200201  7.651579  2.209923

[11229 rows x 4 columns]

In [6]:
# slc_norm_gdf[(slc_norm_gdf['avg_rad'] > 0)]  # include only radiance greater than 0  (np.size = count)
# seg_month_slc_mean = slc_norm_gdf[(slc_norm_gdf['avg_rad'] > 0)].groupby(
#     ['seg_id','ntldate']).agg(
#     {'avg_rad': [np.size, np.mean, np.std]}).reset_index()
# seg_month_slc_mean

# Mean and standard deviation of radiance by Segment and Date - removed radiances <= 0
# Assign custom field names to grouped columns
seg_month_slc_mean = slc_norm_gdf[(slc_norm_gdf['avg_rad'] > 0)].groupby(
    ['seg_id','ntldate']).agg(
    obs_count=('avg_rad', np.size),
    ntl_mean=('avg_rad', np.mean),
    ntl_std=('avg_rad', np.std),
    ).reset_index()
seg_month_slc_mean

,seg_id,ntldate,obs_count,ntl_mean,ntl_std
0,1,20150601,44.0,2.241121,1.427884
1,1,20150701,44.0,2.629657,1.800175
2,1,20150801,44.0,2.463786,1.656322
3,1,20150901,44.0,2.782790,2.041003
4,1,20151001,44.0,4.461700,2.909745
...,...,...,...,...,...
11058,197,20191001,19.0,6.160526,1.687524
11059,197,20191101,19.0,7.576842,1.632779
11060,197,20191201,19.0,8.199474,2.272965
11061,197,20200101,19.0,9.758421,1.827498


In [7]:
type(seg_month_slc_mean)

pandas.core.frame.DataFrame

In [8]:
seg_month_slc_mean.dtypes

seg_id         int64
ntldate       object
obs_count    float64
ntl_mean     float64
ntl_std      float64
dtype: object

In [9]:
# Export normalized mean radiance by segment and month to non-spatial data in Geopackage
with sqlite3.connect(shore_gpkg) as conn:
    seg_month_slc_mean.to_sql('seg_multiple_ntl_mean', conn, if_exists='replace', index=False)

# Pivot Table for Mean Radiance by Segment (row) and Date (column)

In [10]:
# Pivot SHOULD work with grouped output from previous step, but throwing error I couldn't debug
# seg_month_slc_mean.pivot(index=["seg_id"], columns=["ntldate"], values=["ntl_mean"])

# Do it as pivot table where aggregation happens at the same time
slc_norm_gdf['yrmo'] = slc_norm_gdf['ntldate'].str.slice(0,6)
slc_norm_gdf['ntl_'] = slc_norm_gdf['avg_rad']
print(slc_norm_gdf.head())

seg_single_ntl_mean = pd.pivot_table(slc_norm_gdf, values=['ntl_'], index=['seg_id'], columns=['yrmo'],
                    aggfunc={'ntl_': [np.mean, np.std]})
seg_single_ntl_mean

   cf_cvg  shore_ptid      nearest_y ntlsrc   ntldate  avg_rad  seg_id  \
0      14  qrm_633160  476561.890870    slc  20200201     3.86      42   
1      14  qrm_633210  476563.641307    slc  20200201     3.86      42   
2      14  qrm_633260  476573.202228    slc  20200201     3.86      42   
3      14  qrm_633310  476580.967375    slc  20200201     3.86      42   
4      14  qrm_633360  476614.205614    slc  20200201     2.43      42   

   type_geomo     nearest_x                        geometry    yrmo  ntl_  
0  artificial  2.266580e+06  POINT (476561.729 2266579.636)  202002  3.86  
1  artificial  2.266530e+06  POINT (476563.715 2266529.997)  202002  3.86  
2  artificial  2.266483e+06  POINT (476573.146 2266483.337)  202002  3.86  
3  artificial  2.266526e+06  POINT (476581.088 2266526.026)  202002  3.86  
4  artificial  2.266506e+06  POINT (476614.346 2266505.674)  202002  2.43  


ntl_                                                              \
            mean                                                               
yrmo      201506    201507    201508    201509    201510    201511    201512   
seg_id                                                                         
1       2.241121  2.629657  2.463786  2.782790  4.461700  2.422226  2.374236   
2       0.082308 -0.035033 -0.078571  0.015012  0.153258  0.135971  0.041954   
3       0.050231 -0.033034 -0.040655  0.030493  0.205185  0.080632 -0.006766   
4       0.018693  0.010283 -0.049528  0.018237  0.134224  0.061592 -0.019187   
5       0.051036  0.017712 -0.058452 -0.003400  0.096716  0.059856 -0.064398   
...          ...       ...       ...       ...       ...       ...       ...   
193     0.097947 -0.029050 -0.073979  0.015527  0.170451  0.150863  0.003818   
194     0.090921 -0.003253 -0.058849 -0.006725  0.126849  0.128960  0.046184   
195     5.462590  5.485812  5.563566  4.565205  5.114202  4.958255  5.312657   
196     0.108744  0.070679  0.117852  0.082115  0.236973  0.193254  0.057094   
197     5.484439  5.620585  4.185372  5.238149  4.663085  5.644330  5.909269   

                                      ...                                \
                                      ...       std                       
yrmo      201601    201602    201603  ...    201905    201906    201907   
seg_id                                ...                                 
1       2.628433  2.401536  2.642324  ...  1.569650  2.147237  1.836342   
2       0.012048 -0.035729  0.068220  ...  0.023159  0.050034  0.033835   
3      -0.062999 -0.088788  0.067655  ...  0.029876  0.027174  0.031158   
4      -0.070644 -0.088435  0.042337  ...  0.039232  0.045919  0.040005   
5      -0.089211 -0.129416  0.084440  ...  0.024816  0.021272  0.038106   
...          ...       ...       ...  ...       ...       ...       ...   
193    -0.019157 -0.077878  0.014287  ...  0.040812  0.023909  0.026477   
194    -0.031581 -0.056287  0.048855  ...  0.021197  0.047627  0.043553   
195     5.918310  6.651400  5.945987  ...  3.829959  3.636258  3.662762   
196     0.027363  0.069832  0.119725  ...  0.066078  0.010000  0.022721   
197     6.367653  6.037973  6.007597  ...  1.383033  1.019357  1.263320   

                                                                              
                                                                              
yrmo      201908    201909    201910    201911    201912    202001    202002  
seg_id                                                                        
1       1.869998  1.533615  1.433963  1.538892  1.721840  1.695547  1.612189  
2       0.027078  0.031594  0.051924  0.034911  0.022010  0.026727  0.024623  
3       0.041361  0.057862  0.027574  0.036997  0.023260  0.021188  0.020735  
4       0.064681  0.049187  0.042415  0.044120  0.028199  0.042327  0.049186  
5       0.033334  0.028854  0.047275  0.064878  0.034788  0.049660  0.062287  
...          ...       ...       ...       ...       ...       ...       ...  
193     0.043964  0.029466  0.045188  0.030737  0.017210  0.029880  0.023180  
194     0.041641  0.030980  0.049270  0.038820  0.038589  0.028344  0.021994  
195     3.605283  3.091920  3.775965  4.182885  4.664626  4.255520  4.601936  
196     0.009811  0.019621  0.045442  0.005000  0.025788  0.011832  0.037417  
197     1.264528  1.161137  1.687524  1.632779  2.272965  1.827498  2.209923  

[197 rows x 114 columns]

In [11]:
seg_single_ntl_mean.dtypes

            yrmo  
ntl_  mean  201506    float64
            201507    float64
            201508    float64
            201509    float64
            201510    float64
                       ...   
      std   201910    float64
            201911    float64
            201912    float64
            202001    float64
            202002    float64
Length: 114, dtype: object

In [12]:
type(seg_single_ntl_mean)

pandas.core.frame.DataFrame

In [13]:
seg_single_ntl_mean.columns.ravel

<bound method Index.ravel of MultiIndex([('ntl_', 'mean', '201506'),
            ('ntl_', 'mean', '201507'),
            ('ntl_', 'mean', '201508'),
            ('ntl_', 'mean', '201509'),
            ('ntl_', 'mean', '201510'),
            ('ntl_', 'mean', '201511'),
            ('ntl_', 'mean', '201512'),
            ('ntl_', 'mean', '201601'),
            ('ntl_', 'mean', '201602'),
            ('ntl_', 'mean', '201603'),
            ...
            ('ntl_',  'std', '201905'),
            ('ntl_',  'std', '201906'),
            ('ntl_',  'std', '201907'),
            ('ntl_',  'std', '201908'),
            ('ntl_',  'std', '201909'),
            ('ntl_',  'std', '201910'),
            ('ntl_',  'std', '201911'),
            ('ntl_',  'std', '201912'),
            ('ntl_',  'std', '202001'),
            ('ntl_',  'std', '202002')],
           names=[None, None, 'yrmo'], length=114)>

In [14]:
# concatenate aggregate columns into a single column name
# If you use an underscore to join, it joins all characters instead of each word (string in the tuple)
seg_single_ntl_mean.columns = [''.join(x) for x in seg_single_ntl_mean.columns.ravel()]
seg_single_ntl_mean = seg_single_ntl_mean.reset_index()
seg_single_ntl_mean

,seg_id,ntl_mean201506,ntl_mean201507,ntl_mean201508,ntl_mean201509,ntl_mean201510,ntl_mean201511,ntl_mean201512,ntl_mean201601,ntl_mean201602,...,ntl_std201905,ntl_std201906,ntl_std201907,ntl_std201908,ntl_std201909,ntl_std201910,ntl_std201911,ntl_std201912,ntl_std202001,ntl_std202002
0,1,2.241121,2.629657,2.463786,2.782790,4.461700,2.422226,2.374236,2.628433,2.401536,...,1.569650,2.147237,1.836342,1.869998,1.533615,1.433963,1.538892,1.721840,1.695547,1.612189
1,2,0.082308,-0.035033,-0.078571,0.015012,0.153258,0.135971,0.041954,0.012048,-0.035729,...,0.023159,0.050034,0.033835,0.027078,0.031594,0.051924,0.034911,0.022010,0.026727,0.024623
2,3,0.050231,-0.033034,-0.040655,0.030493,0.205185,0.080632,-0.006766,-0.062999,-0.088788,...,0.029876,0.027174,0.031158,0.041361,0.057862,0.027574,0.036997,0.023260,0.021188,0.020735
3,4,0.018693,0.010283,-0.049528,0.018237,0.134224,0.061592,-0.019187,-0.070644,-0.088435,...,0.039232,0.045919,0.040005,0.064681,0.049187,0.042415,0.044120,0.028199,0.042327,0.049186
4,5,0.051036,0.017712,-0.058452,-0.003400,0.096716,0.059856,-0.064398,-0.089211,-0.129416,...,0.024816,0.021272,0.038106,0.033334,0.028854,0.047275,0.064878,0.034788,0.049660,0.062287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,193,0.097947,-0.029050,-0.073979,0.015527,0.170451,0.150863,0.003818,-0.019157,-0.077878,...,0.040812,0.023909,0.026477,0.043964,0.029466,0.045188,0.030737,0.017210,0.029880,0.023180
193,194,0.090921,-0.003253,-0.058849,-0.006725,0.126849,0.128960,0.046184,-0.031581,-0.056287,...,0.021197,0.047627,0.043553,0.041641,0.030980,0.049270,0.038820,0.038589,0.028344,0.021994
194,195,5.462590,5.485812,5.563566,4.565205,5.114202,4.958255,5.312657,5.918310,6.651400,...,3.829959,3.636258,3.662762,3.605283,3.091920,3.775965,4.182885,4.664626,4.255520,4.601936
195,196,0.108744,0.070679,0.117852,0.082115,0.236973,0.193254,0.057094,0.027363,0.069832,...,0.066078,0.010000,0.022721,0.009811,0.019621,0.045442,0.005000,0.025788,0.011832,0.037417


In [15]:
# Export single segment mean radiance by month to non-spatial data in Geopackage
with sqlite3.connect(shore_gpkg) as conn:
    seg_single_ntl_mean.to_sql('seg_single_ntl_mean', conn, if_exists='replace', index=False)

# Shoreline Points and Segments to join with summary tables

In [16]:
# Import shoreline points and shoreline segments
shore_segments_lyr = 'shoreQR_segments'
shore_points_lyr = 'shoreQR_50m_pts'
shore_segments_gdf = gpd.read_file(shore_gpkg, layer=shore_segments_lyr)
shore_points_gdf = gpd.read_file(shore_gpkg, layer=shore_points_lyr)
print(shore_segments_gdf.dtypes)
print(shore_points_gdf.dtypes)

type_geomorph      object
length_km         float64
seg_id              int64
shore_desc         object
desc_abbrev        object
geometry         geometry
dtype: object
shore_desc         object
desc_abbrev        object
distance          float64
shore_ptid         object
type_geomorph      object
seg_id              int64
nearest_x         float64
nearest_y         float64
geometry         geometry
dtype: object


## 50m points per segment ID

In [17]:
# Count of points per segment
pts_per_segment_df = shore_points_gdf.groupby(
    ['seg_id']).agg(
    pt_count=('shore_ptid', np.size)).reset_index()
pts_per_segment_df

,seg_id,pt_count
0,1,44
1,2,75
2,3,115
3,4,113
4,5,43
...,...,...
192,193,101
193,194,164
194,195,221
195,196,16


## Unique NTL pixels by Segment (using unique radiance values as proxy)

In [18]:
# Count Unique values of Radiance by segment/date
grouped_seg_date_rad_df = slc_norm_gdf.groupby(
    ['seg_id', 'ntldate', 'avg_rad']).agg(
    unique_rad_count=('seg_id', np.size),).reset_index()
grouped_seg_date_rad_df

# # Example with one segment / date combo
# grouped_seg_date_rad_df = slc_norm_gdf[(slc_norm_gdf['seg_id'] == 1) & (slc_norm_gdf['ntldate'] == '20200201')].groupby(
#     ['seg_id', 'ntldate', 'avg_rad']).agg(
#     unique_rad_count=('seg_id', np.size),).reset_index()
# grouped_seg_date_rad_df

,seg_id,ntldate,avg_rad,unique_rad_count
0,1,20150601,0.491468,1
1,1,20150601,0.859881,10
2,1,20150601,0.975873,8
3,1,20150601,1.866755,6
4,1,20150601,2.362613,3
...,...,...,...,...
145659,197,20200101,12.310000,3
145660,197,20200201,3.860000,1
145661,197,20200201,5.730000,8
145662,197,20200201,9.010000,7


In [19]:
# Number of pixels (unique radiance values) by segment/date
pixels_per_seg_date_df = grouped_seg_date_rad_df.groupby(
    ['seg_id', 'ntldate']).agg(
    rad_pixels=('seg_id', np.size),).reset_index()
pixels_per_seg_date_df

,seg_id,ntldate,rad_pixels
0,1,20150601,8
1,1,20150701,8
2,1,20150801,8
3,1,20150901,8
4,1,20151001,8
...,...,...,...
11224,197,20191001,4
11225,197,20191101,4
11226,197,20191201,4
11227,197,20200101,4


In [20]:
# Number of pixels by segment
# There are some segment/dates with a differing number of pixels - take the max 
#   assume that at some date, all pixels have valid values and that they are unique for each pixel
pixels_per_seg_df = pixels_per_seg_date_df.groupby(
    ['seg_id']).agg(
    ntl_pixels=('rad_pixels', np.max)).reset_index()
pixels_per_seg_df

,seg_id,ntl_pixels
0,1,8
1,2,11
2,3,16
3,4,18
4,5,6
...,...,...
192,193,14
193,194,24
194,195,29
195,196,3


# Join attributes for each segment & export to Geopackage

In [21]:
# Join data frames -- segments, pts/segment, pixels/segment, radiance mean & std dev by month
dfs = [df.set_index(['seg_id']) for df in [shore_segments_gdf, pts_per_segment_df, pixels_per_seg_df, seg_single_ntl_mean]]
segment_ntlatts_gdf = pd.concat(dfs, axis=1).reset_index()

In [22]:
type(segment_ntlatts_gdf)

geopandas.geodataframe.GeoDataFrame

In [23]:
segment_ntlatts_gdf

,seg_id,type_geomorph,length_km,shore_desc,desc_abbrev,geometry,pt_count,ntl_pixels,ntl_mean201506,ntl_mean201507,...,ntl_std201905,ntl_std201906,ntl_std201907,ntl_std201908,ntl_std201909,ntl_std201910,ntl_std201911,ntl_std201912,ntl_std202001,ntl_std202002
0,1,cliff or rocky,2.188708,Isla Mujeres,imu,"MULTILINESTRING ((530027.596 2344472.852, 5300...",44,8,2.241121,2.629657,...,1.569650,2.147237,1.836342,1.869998,1.533615,1.433963,1.538892,1.721840,1.695547,1.612189
1,2,sand,3.738697,Isla Contoy,ico,"MULTILINESTRING ((521010.617 2378995.910, 5210...",75,11,0.082308,-0.035033,...,0.023159,0.050034,0.033835,0.027078,0.031594,0.051924,0.034911,0.022010,0.026727,0.024623
2,3,mixed mud and sand,5.758523,QR mainland,qrm,"MULTILINESTRING ((439281.990 2130747.294, 4392...",115,16,0.050231,-0.033034,...,0.029876,0.027174,0.031158,0.041361,0.057862,0.027574,0.036997,0.023260,0.021188,0.020735
3,4,sand,5.622333,QR mainland,qrm,"MULTILINESTRING ((433485.994 2090201.431, 4334...",113,18,0.018693,0.010283,...,0.039232,0.045919,0.040005,0.064681,0.049187,0.042415,0.044120,0.028199,0.042327,0.049186
4,5,sand,2.136723,QR mainland,qrm,"MULTILINESTRING ((410987.143 2013924.228, 4109...",43,6,0.051036,0.017712,...,0.024816,0.021272,0.038106,0.033334,0.028854,0.047275,0.064878,0.034788,0.049660,0.062287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,193,cliff or rocky,5.019248,QR mainland,qrm,"MULTILINESTRING ((444590.092 2376499.228, 4447...",101,14,0.097947,-0.029050,...,0.040812,0.023909,0.026477,0.043964,0.029466,0.045188,0.030737,0.017210,0.029880,0.023180
193,194,sand,8.177061,Holbox,hol,"MULTILINESTRING ((488231.217 2388467.313, 4882...",164,24,0.090921,-0.003253,...,0.021197,0.047627,0.043553,0.041641,0.030980,0.049270,0.038820,0.038589,0.028344,0.021994
194,195,sand,11.078166,QR mainland,qrm,"MULTILINESTRING ((516830.778 2314283.226, 5168...",221,29,5.462590,5.485812,...,3.829959,3.636258,3.662762,3.605283,3.091920,3.775965,4.182885,4.664626,4.255520,4.601936
195,196,sand,0.784313,Isla de la Pasion,ipa,"MULTILINESTRING ((513991.173 2272135.231, 5139...",16,3,0.108744,0.070679,...,0.066078,0.010000,0.022721,0.009811,0.019621,0.045442,0.005000,0.025788,0.011832,0.037417


In [24]:
# Export segments with Nightime lights attributes
segment_ntlatts_gdf.to_file(shore_gpkg, layer='shoreQR_segments_ntl', driver="GPKG")
